# SuSIE results post process

This notebook is to post-process the susie results into different text file


In [ ]:
[global]
import glob
import pandas as pd
# Input
parameter: rds_path = paths
# For complete susie, region_list or tad_list, for susie_rss , LD region list 
parameter: region_list = path

# A region list file documenting the chr_pos_ref_alt of a susie_object
parameter: cwd = path("output")
parameter: name = "demo"
region_tbl = pd.read_csv(region_list,sep = "\t")

## Path to work directory where output locates
## Containers that contains the necessary packages
parameter: container = ""
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20

# use this function to edit memory string for PLINK input
from sos.utils import expand_size

In [ ]:
[susie_to_tsv]
input: rds_path, group_by = 1
output: f"{cwd}/{_input:bn}.lbf.tsv",f"{cwd}/{_input:bn}.variant.tsv",f"{cwd}/{_input:bn}.effect.tsv"
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container
    library("dplyr")
    library("tibble")
    library("purrr")
    library("readr")
    library("stringr")
    
    extract_lbf = function(susie_obj){
    
    if("variants" %in% names(susie_obj) ){
    ss_bf = tibble(snps = susie_obj$variants%>%str_replace("_",":"), cs_index = ifelse(is.null(susie_obj$sets$cs_index), 0, paste0(susie_obj$sets$cs_index,collapse =",")),names = "LD1" )
    }
      else 
      {
    ss_bf = tibble(snps = susie_obj$variable_name, cs_index = ifelse(is.null(susie_obj$sets$cs_index), 0, paste0(susie_obj$sets$cs_index,collapse =",")),names = ${f'{_input:bnnn}'})
     }
    
    ss_bf = ss_bf%>%cbind(susie_obj$lbf_variable%>%t)%>%as_tibble()
    
    return(ss_bf)
    }
    
    extract_variants_pip = function(susie_obj,region_list){
    susie_tb = tibble( variants =  names(susie_obj$pip)[which( susie_obj$pip >= 0)], 
                           snps_index = which(( susie_obj$pip >= 0))) %>% 
        mutate(chromosome = map_chr(variants, ~read.table(text = .x, sep = ":")$V1%>%str_replace("chr","") ),
                position  = map_chr(variants, ~read.table(text = .x, sep = ":")$V2  ),
                ref = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V2  ),
                alt = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V3  ),
                position  = map_dbl(position , ~read.table(text = .x, sep = "_",as.is = T)$V1  )
                             )%>% mutate(cs_order =map(snps_index , ~tryCatch(which(pmap(list( a= susie_obj$sets$cs) , function(a) .x %in% a )%>%unlist()), error = function(e) return(F) )  )%>%as.numeric%>%replace_na(0),
                             cs_id = map_chr(cs_order,~ifelse(.x ==0, "None" ,names(susie_obj$sets$cs)[.x] ) ),
                             log10_base_factor = map_chr(snps_index,~paste0( susie_obj$lbf_variable[,.x],  collapse = ";")),
                             pip = map_dbl(snps_index,~( susie_obj$pip[.x])),
                             posterior_mean = map_dbl(snps_index,~( susie_obj$coef[.x])),
                             posterior_sd = map_dbl(snps_index,~(susie_get_posterior_sd(susie_obj)[.x])),
                             z = posterior_mean/posterior_sd)
  
          susie_tb =  susie_tb%>%mutate(  molecular_trait_id = region_list$molecular_trait_id,
                             finemapped_region_start = region_list$finemapped_region_start,
                             finemapped_region_end = region_list$finemapped_region_end)
              }
          return(susie_tb)
        }
  
  
  
  

     extract_effect_pip = function(susie_obj,region_list){
      susie_effect_tb_ls = region_list%>%mutate(result_tb = list( tibble(phenotype = susie_obj$name,
        V = susie_obj$V,effect_id = paste0("L",1:length(V) ) ,
        cs_log10bf = susie_obj$lbf,
        cs_min_r2  = (susie_obj$sets$purity[effect_id,1])%>%as.numeric%>%replace_na(0),
        cs_avg_r2  = (susie_obj$sets$purity[effect_id,2])%>%as.numeric%>%replace_na(0),
        coverage = map(effect_id, ~susie_obj$sets$coverage[which(names(susie_obj$sets$cs) == .x )])%>%as.numeric%>%replace_na(0),
        cs = map_chr(susie_obj$sets$cs[effect_id],~susie_tb$variants[.x]%>%paste0(collapse = ";"))) ))%>%unnest(result_tb)
        susie_effect_tb_ls$cs[susie_effect_tb_ls$cs == ""] = "None"
      return(susie_effect_tb_ls)
      }
      } 
  
    
  
  
    susie_obj = readRDS(rds_path)
    lbf = extract_lbf(susie_obj)
    region_list = read_delim("${region_list}","\t")
    if(ncol(region_list) == 3 ){   region_list =  region_list%>%mutate(`#chr` = `#chr`%>%str_remove_all(" ") , ID = paste0(`#chr`,"_",start,"_",stop) ) } # LD_list 
    if(region_list$start[1] - region_list$end[1]  == -1 ){ 
        region_list = region_list%>%mutate( start = start - ${windows} ,end = start +${windows}) # region_list for fix cis windows  
          } 
      if("gene_id" %in% colnames(region_list)){region_list = region_list%>%mutate(ID = gene_id)  } # region_list for gene
    region_list = region_list%>%select(molecular_trait_id = ID, chromosome  = `#chr`,finemapped_region_start = start ,finemapped_region_end = end)  # Formatting
    variants_pip = extract_variants_pip( susie_obj , region_list)
    effect_pip = extract_effect_pip( susie_obj , region_list)
    lbf%>%write_delim("${_output[0]}","\t")
    variants_pip%>%write_delim("${_output[1]}","\t")
    effect_pip%>%write_delim("${_output[2]}","\t")

In [ ]:
[susie_to_tsv]
parameter: region_list = path()
parameter: windows = 1000000
parameter: cwd = path("output")
input: rds_path, group_by = 1
# example input: demo.ENSG00000197375.unisusie.fit.ALL.rds
output: f"{cwd}/{_input:bn}.effect.tsv",f"{cwd}/{_input:bn}.variant.tsv",f"{cwd}/{_input:bn}.lbf.tsv"
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container
    library("susieR")
    library("dplyr")
    library("tibble")
    library("purrr")
    library("readr")
    library("tidyr")
    library("stringr")
    
    extract_lbf = function(susie_obj){
    
    if("variants" %in% names(susie_obj) ){
    ss_bf = tibble(snps = susie_obj$variants%>%str_replace("_",":"), cs_index = ifelse(is.null(susie_obj$sets$cs_index), 0, paste0(susie_obj$sets$cs_index,collapse =",")),names = "LD1" )
    }
      else 
      {
    ss_bf = tibble(snps = susie_obj$variable_name, cs_index = ifelse(is.null(susie_obj$sets$cs_index), 0, paste0(susie_obj$sets$cs_index,collapse =",")),names = ${_name})
     }
    
    ss_bf = ss_bf%>%cbind(susie_obj$lbf_variable%>%t)%>%as_tibble()
    
    return(ss_bf)
    }
    
    
  
  
  
  
  
  
    # Get list of cs snps
    rds_path = ${_input:r}
    molecular_trait_id = ${_name}
    tad_list = read_delim("${region_list}","\t")
    tad_list = tad_list%>%filter(gene_id == molecular_trait_id )
    if("tad_index" %in% colnames(tad_list) ){
          tad_list = tad_list%>%select(molecular_trait_id = gene_id, chromosome  = `#chr`, finemapped_region_start = start_x ,finemapped_region_end = end_x)
                                       } else {
          tad_list = tad_list%>%mutate("chromosome" =  `#chr`,
                              molecular_trait_id = gene_id , 
                              finemapped_region_end = start+${windows}+1 )  
        }
    susie_tb_ls = list()
    susie_effect_tb_ls = list()
    susie_list = readRDS(rds_path)
    
    if(!is.null(susie_list$sets$cs)){
        ## Get variant table 
        susie_tb = tibble( variants =  names(susie_list$pip)[which( susie_list$pip >= 0)], 
                           snps_index = which(( susie_list$pip >= 0))) %>% 
        mutate(chromosome = map_chr(variants, ~read.table(text = .x, sep = ":")$V1%>%str_replace("chr","") ),
                position  = map_chr(variants, ~read.table(text = .x, sep = ":")$V2  ),
                ref = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V2  ),
                alt = map_chr(position , ~read.table(text = .x, sep = "_",colClasses = "character")$V3  ),
                position  = map_dbl(position , ~read.table(text = .x, sep = "_",as.is = T)$V1  )
                             )
           
        susie_tb_ls = susie_tb %>% mutate(cs_order =map(snps_index , ~tryCatch(which(pmap(list( a= susie_list$sets$cs) , function(a) .x %in% a )%>%unlist()), error = function(e) return(F) )  )%>%as.numeric%>%replace_na(0),
                             cs_id = map_chr(cs_order,~ifelse(.x ==0, "None" ,names(susie_list$sets$cs)[.x] ) ),
                             log10_base_factor = map_chr(snps_index,~paste0( susie_list$lbf_variable[,.x],  collapse = ";")),
                             pip = map_dbl(snps_index,~( susie_list$pip[.x])),
                             posterior_mean = map_dbl(snps_index,~( susie_list$coef[.x])),
                             posterior_sd = map_dbl(snps_index,~(susie_get_posterior_sd(susie_list)[.x])),
                             z = posterior_mean/posterior_sd)%>%
                    mutate(  molecular_trait_id = tad_list$molecular_trait_id,
                             finemapped_region_start = tad_list$finemapped_region_start,
                             finemapped_region_end = tad_list$finemapped_region_end)
        write_delim(susie_tb_ls,paste0("${_output[1]:nnnn}.",susie_list$name,".variant.tsv"),"\t")                                                                     
                                                                                                                                                                           
       ## Get effect table
      ## I make use of susie_list$name to catch the tissue name. This attribute is added in the previous step. But
      ## it's not in all susie objects. Maybe we can use other strategy? eg. extract it from file name
                                                                                     
        susie_effect_tb_ls = tad_list%>%mutate(result_tb = list( tibble(phenotype = susie_list$name,
        V = susie_list$V,effect_id = paste0("L",1:length(V) ) ,
        cs_log10bf = susie_list$lbf,
        cs_min_r2  = (susie_list$sets$purity[effect_id,1])%>%as.numeric%>%replace_na(0),
        cs_avg_r2  = (susie_list$sets$purity[effect_id,2])%>%as.numeric%>%replace_na(0),
        coverage = map(effect_id, ~susie_list$sets$coverage[which(names(susie_list$sets$cs) == .x )])%>%as.numeric%>%replace_na(0),
        cs = map_chr(susie_list$sets$cs[effect_id],~susie_tb$variants[.x]%>%paste0(collapse = ";"))) ))%>%unnest(result_tb)
        susie_effect_tb_ls$cs[susie_effect_tb_ls$cs == ""] = "None"
        susie_effect_tb_ls %>% write_delim(paste0("${_output[0]:nnnn}.",susie_list$name,".effect.tsv"),"\t") 
    } 
  
bash: expand = '${ }', stdout = f"{_output[0]:nn}.stdout", stderr = f"{_output[0]:nn}.stderr", container = container
    for i in ${_output:nr}; do
    bgzip $i;
    done